In [1]:
import numpy as np
import pandas as pd

import logistic_regression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

# Aids

In [16]:
aids = pd.read_csv('data/aids.csv')

y = aids['target']
X = aids.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [24]:
lr = logistic_regression.LogisticRegressor()

lr.fit(X, y)
lr.accuracy(X, y)

0.7568957456755493

In [25]:
sklearn_lr = LogisticRegression(max_iter=1000, tol=1e-3, solver='lbfgs')
sklearn_lr.fit(X, y)
sklearn_lr.score(X, y)
y_pred = sklearn_lr.predict(X)

In [26]:
print(f"Our implementation\n{lr.confusion_matrix(X, y)}")
print(f"Sklearn implementation\n{confusion_matrix(y, y_pred)}")

Our implementation
[[   1    0]
 [ 520 1618]]
Sklearn implementation
[[1528   90]
 [ 207  314]]
